# Adaptive Boosting Classifiers
---

## Introduction

The second models we are going to use to try tackling the current problem are **adaptive boosting classifiers**.

As always, the prerequisite step consists on loading the appropriate packages to perform our work:

In [1]:
# Activate 'airbnb' environment:
!source activate airbnb

In [2]:
# Needed packages:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.externals import joblib
from utils import (create_training_testing_datasets,
                   calculate_dcg,
                   calculate_ndcg,
                   clf_prediction,
                   ndcg_mean_score_calc,
                   detailed_ndcg_mean_score_calc)

/Applications/anaconda3/envs/airbnb/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


---

## Create training and testing datasets

In [3]:
# Load the data:
consolidated_dataset = pd.read_csv("../data/consolidated_dataset.csv")

# Check basic info:
print("*** Some basic info:")
print("'consolidated_dataset' has {} data points with {} variables each.".format(*consolidated_dataset.shape))
print("'consolidated_dataset' counts {} missing values.".format(consolidated_dataset.isnull().sum().sum()))

# Give a look to the first lines:
print("\n*** First lines:")
display(consolidated_dataset.head())

*** Some basic info:
'consolidated_dataset' has 213451 data points with 161 variables each.
'consolidated_dataset' counts 0 missing values.

*** First lines:


,age,country_destination,nans,day_account_created,weekday_account_created,week_account_created,month_account_created,year_account_created,day_first_active,weekday_first_active,...,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,-1.0,NDF,1.225078,28,0,26,6,2010,19,3,...,0,0,0,0,0,0,0,0,0,0
1,38.0,NDF,-0.453135,25,2,21,5,2011,23,5,...,0,0,0,0,0,0,0,0,0,0
2,56.0,US,-0.453135,28,1,39,9,2010,9,1,...,0,0,0,0,0,0,0,0,0,0
3,42.0,other,-0.453135,5,0,49,12,2011,31,5,...,0,0,0,0,0,0,0,0,0,0
4,41.0,US,0.385972,14,1,37,9,2010,8,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Create training and testing datasets:
X_train, X_test, y_train, y_test, encoding_dict = create_training_testing_datasets(consolidated_dataset)

---

## Calculate Normalized DCG scores

### "Out-of-the-box" adaptive boosting classifier

In [5]:
# Initialize the classifier:
ab_clf = AdaBoostClassifier(random_state=42)

# Train the classifier:
print("Time info about classifier training:")
%time ootb_ab_clf = ab_clf.fit(X_train, y_train)

Time info about classifier training:
CPU times: user 50.2 s, sys: 1.96 s, total: 52.2 s
Wall time: 43 s


In [6]:
# Perform one prediction to check classifier:
print("Time info about classifier prediction:")
%time preds_list = clf_prediction(ootb_ab_clf, X_train[0])
print("***")

# Reverse encoding dictionary:
decoding_dict = dict(map(reversed, encoding_dict.items()))

# Print result:
print("For the classifier check:")
print("- Real first booking destination country: {}".format(decoding_dict[y_train[0]]))
print("- Predictions list: {}".format([decoding_dict[x] for x in preds_list]))

Time info about classifier prediction:
CPU times: user 12.2 ms, sys: 2.4 ms, total: 14.6 ms
Wall time: 13.1 ms
***
For the classifier check:
- Real first booking destination country: US
- Predictions list: ['US', 'NDF', 'other', 'FR', 'IT', 'GB', 'ES', 'CA', 'NL', 'DE', 'AU', 'PT']


In [7]:
# Calculate nDCG mean score on training dataset:
print("Time info about nDCG mean score calculation on training dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(ootb_ab_clf, X_train, y_train)
print("***")
print("On training dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on training dataset:
CPU times: user 18min 24s, sys: 11.3 s, total: 18min 35s
Wall time: 18min 29s
***
On training dataset, classifier nDCG mean score is 0.823032.


In [8]:
# Calculate nDCG mean score on testing dataset:
print("Time info about nDCG mean score calculation on testing dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(ootb_ab_clf, X_test, y_test)
print("***")
print("On testing dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on testing dataset:
CPU times: user 4min 36s, sys: 3.57 s, total: 4min 39s
Wall time: 4min 37s
***
On testing dataset, classifier nDCG mean score is 0.822064.


In [9]:
# Calculate nDCG mean score for each class on testing dataset:
print("Time info about nDCG mean score calculation for each class on testing dataset:")
%time ndcg_mean_scores_list = detailed_ndcg_mean_score_calc(ootb_ab_clf, X_test, y_test, encoding_dict)
print("***")
print("Detailed results for each class on testing dataset:")
for country_dest in range(12):
    print("nDCG mean score for {}: {:.6f}".format(decoding_dict[country_dest],
                                                  ndcg_mean_scores_list[country_dest]))

Time info about nDCG mean score calculation for each class on testing dataset:
CPU times: user 4min 34s, sys: 2.91 s, total: 4min 37s
Wall time: 4min 35s
***
Detailed results for each class on testing dataset:
nDCG mean score for AU: 0.000000
nDCG mean score for CA: 0.016691
nDCG mean score for DE: 0.014134
nDCG mean score for ES: 0.087411
nDCG mean score for FR: 0.434455
nDCG mean score for GB: 0.073211
nDCG mean score for IT: 0.243626
nDCG mean score for NDF: 0.948951
nDCG mean score for NL: 0.002545
nDCG mean score for PT: 0.000000
nDCG mean score for US: 0.785494
nDCG mean score for other: 0.496264


In [10]:
# Save model:
joblib.dump(ootb_ab_clf, "../models/ootb_ab_clf.pkl")

['../models/ootb_ab_clf.pkl']

### "Optimized" adaptive boosting classifier

In [11]:
# Set parameters of the random grid:
n_estimators = [int(x) for x in np.linspace(10, 100, num=10)]
learning_rate = [1., 0.1, 0.01, 0.001, 0.0001, 0.00001]
random_state = [42]

# Create the random grid:
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'random_state': random_state}

# Perform randomized search cross validation:

ab_clf = AdaBoostClassifier()

ab_clf_random = RandomizedSearchCV(estimator=ab_clf,
                                   param_distributions=random_grid,
                                   n_iter=20,
                                   n_jobs=-1,
                                   random_state=42,
                                   verbose=0)

print("Time info about randomized search cross validation:")
%time ab_clf_fit = ab_clf_random.fit(X_train, y_train)

# Get the "best" classifier:
opt_ab_clf = ab_clf_fit.best_estimator_

Time info about randomized search cross validation:


/Applications/anaconda3/envs/airbnb/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 1min 10s, sys: 3.21 s, total: 1min 13s
Wall time: 17min 9s


In [12]:
# Parameters of the "best" classifier:
print("*** Parameters of the 'best' classifier:")
for param, param_value in ab_clf_fit.best_params_.items():
    print("- Paramater {}: {}".format(param, param_value))

*** Parameters of the 'best' classifier:
- Paramater random_state: 42
- Paramater n_estimators: 70
- Paramater learning_rate: 1.0


In [13]:
# Perform one prediction to check classifier:
print("Time info about classifier prediction:")
%time preds_list = clf_prediction(opt_ab_clf, X_train[0])
print("***")

# Reverse encoding dictionary:
decoding_dict = dict(map(reversed, encoding_dict.items()))

# Print result:
print("For the classifier check:")
print("- Real first booking destination country: {}".format(decoding_dict[y_train[0]]))
print("- Predictions list: {}".format([decoding_dict[x] for x in preds_list]))

Time info about classifier prediction:
CPU times: user 17.2 ms, sys: 4.69 ms, total: 21.9 ms
Wall time: 18.5 ms
***
For the classifier check:
- Real first booking destination country: US
- Predictions list: ['US', 'NDF', 'other', 'FR', 'IT', 'GB', 'ES', 'CA', 'NL', 'DE', 'AU', 'PT']


In [14]:
# Calculate nDCG mean score on training dataset:
print("Time info about nDCG mean score calculation on training dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(opt_ab_clf, X_train, y_train)
print("***")
print("On training dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on training dataset:
CPU times: user 25min 43s, sys: 16.1 s, total: 25min 59s
Wall time: 25min 50s
***
On training dataset, classifier nDCG mean score is 0.823412.


In [15]:
# Calculate nDCG mean score on testing dataset:
print("Time info about nDCG mean score calculation on testing dataset:")
%time ndcg_mean_score = ndcg_mean_score_calc(opt_ab_clf, X_test, y_test)
print("***")
print("On testing dataset, classifier nDCG mean score is {:.6f}.".format(ndcg_mean_score))

Time info about nDCG mean score calculation on testing dataset:
CPU times: user 6min 23s, sys: 4.3 s, total: 6min 28s
Wall time: 6min 25s
***
On testing dataset, classifier nDCG mean score is 0.821829.


In [16]:
# Calculate nDCG mean score for each class on testing dataset:
print("Time info about nDCG mean score calculation for each class on testing dataset:")
%time ndcg_mean_scores_list = detailed_ndcg_mean_score_calc(opt_ab_clf, X_test, y_test, encoding_dict)
print("***")
print("Detailed results for each class on testing dataset:")
for country_dest in range(12):
    print("nDCG mean score for {}: {:.6f}".format(decoding_dict[country_dest],
                                                  ndcg_mean_scores_list[country_dest]))

Time info about nDCG mean score calculation for each class on testing dataset:
CPU times: user 6min 23s, sys: 4.46 s, total: 6min 27s
Wall time: 6min 24s
***
Detailed results for each class on testing dataset:
nDCG mean score for AU: 0.000000
nDCG mean score for CA: 0.017891
nDCG mean score for DE: 0.012309
nDCG mean score for ES: 0.088466
nDCG mean score for FR: 0.434981
nDCG mean score for GB: 0.067387
nDCG mean score for IT: 0.233823
nDCG mean score for NDF: 0.949711
nDCG mean score for NL: 0.002545
nDCG mean score for PT: 0.000000
nDCG mean score for US: 0.783837
nDCG mean score for other: 0.495794


In [17]:
# Save model:
joblib.dump(opt_ab_clf, "../models/opt_ab_clf.pkl")

['../models/opt_ab_clf.pkl']

---

## Conclusion

As a quick conclusion, for the results obtained for the last "optimized" adaptive boosting classifier, we can note 4 major elements:
* On testing dataset, it gets a better nDCG mean score than the one obtained by the naive model.
* On testing dataset, it gets better nDCG mean scores for predicting correctly Canada, Germany, Spain, France, Great Britain, Netherlands and USA than the ones obtained by the naive model.
* On testing dataset, it gets worse nDCG mean scores for predicting correctly Italy, no destination found and other than the ones obtained by the naive model.
* On testing dataset, it is as "bad" as the naive model for predicting correctly Australia and Portugal.